<a href="https://colab.research.google.com/github/Ersel26/Dogs-and-cats-detection/blob/main/Dogs_and_cats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

In [8]:
DATADIR = r"/content/drive/MyDrive/Colab Notebooks/cats-and-dogs/PetImages"

CATEGORIES = ["Cat", "Dog"]

In [9]:
IMG_SIZE = 75
training_data = []

def create_train_data(training_data, IMG_SIZE):
    for c in CATEGORIES:
        path = path = DATADIR + "/" + c
        class_name = CATEGORIES.index(c)
        for img in os.listdir(path):
            
            try:
                img_arr = cv2.imread(
                    os.path.join(path, img),
                    cv2.IMREAD_GRAYSCALE)
                img_arr = cv2.resize(img_arr, (IMG_SIZE, IMG_SIZE))

                data = [img_arr, class_name]
                training_data.append(data)
            except Exception as e:
                pass  

In [10]:
create_train_data(training_data, IMG_SIZE)

In [11]:
import random

random.shuffle(training_data)

In [30]:
X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y = np.array(y)

X = X/255.0

In [31]:
from sklearn.model_selection import train_test_split

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=101)

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

In [34]:
model = Sequential()

model.add(Conv2D(filters = 256,
                kernel_size = (3,3),
                strides = (1,1),
                padding = "same",
                activation = "relu",
                input_shape=X.shape[1:]))

model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(filters = 256,
                kernel_size = (3,3),
                strides = (1,1),
                padding = "same",
                activation = "relu",
                input_shape=X.shape[1:]))

model.add(MaxPooling2D(pool_size = (2,2)))


model.add(Flatten())


model.add(Dense(units = 64,
               activation = "relu"))

model.add(Dropout(rate = 0.3))

model.add(Dense(units = 32,
               activation = "relu"))

model.add(Dropout(rate = 0.3))

model.add(Dense(units = 16,
               activation = "relu"))

model.add(Dropout(rate = 0.3))

model.add(Dense(units = 1,
               activation = "sigmoid"))

In [35]:
model.compile(
    optimizer = "adam",
    loss = "binary_crossentropy",
    metrics = ["accuracy"]
)

In [18]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor = "val_loss", mode = "min", verbose = 1, patience = 5)

In [36]:
model.fit(
    x = X_train,
    y = y_train,
    batch_size = 128,
    epochs = 25,
    validation_split = 0.15,
    callbacks = [early_stop]
)

Epoch 1/25
133/133 [==============================] - 32s 161ms/step - loss: 0.6932 - accuracy: 0.5121 - val_loss: 0.6867 - val_accuracy: 0.5494
Epoch 2/25
133/133 [==============================] - 19s 147ms/step - loss: 0.6658 - accuracy: 0.6004 - val_loss: 0.6416 - val_accuracy: 0.6552
Epoch 3/25
133/133 [==============================] - 20s 147ms/step - loss: 0.6150 - accuracy: 0.6739 - val_loss: 0.5795 - val_accuracy: 0.7056
Epoch 4/25
133/133 [==============================] - 20s 148ms/step - loss: 0.5679 - accuracy: 0.7181 - val_loss: 0.5373 - val_accuracy: 0.7286
Epoch 5/25
133/133 [==============================] - 20s 149ms/step - loss: 0.5180 - accuracy: 0.7525 - val_loss: 0.4971 - val_accuracy: 0.7560
Epoch 6/25
133/133 [==============================] - 20s 148ms/step - loss: 0.4685 - accuracy: 0.7843 - val_loss: 0.5348 - val_accuracy: 0.7243
Epoch 7/25
133/133 [==============================] - 20s 148ms/step - loss: 0.4224 - accuracy: 0.8140 - val_loss: 0.5079 - val_ac

In [40]:
predictions = (model.predict(X_test) > 0.5).astype("int32")

157/157 [==============================] - 2s 13ms/step


In [38]:
from sklearn.metrics import classification_report, confusion_matrix

In [41]:
print(classification_report(y_test, predictions))
print("\n")
print(confusion_matrix(y_test, predictions))

              precision    recall  f1-score   support

           0       0.77      0.77      0.77      2476
           1       0.78      0.77      0.77      2518

    accuracy                           0.77      4994
   macro avg       0.77      0.77      0.77      4994
weighted avg       0.77      0.77      0.77      4994



[[1918  558]
 [ 582 1936]]
